# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.296257     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.391867     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [8]:
model_all.coefficients.sort('value',ascending=False)

name,index,value,stderr
(intercept),None,274873.0559504957,None
bathrooms,None,8468.531086910072,None
grade,None,842.0680348976282,None
sqft_living_sqrt,None,350.0605533860648,None
sqft_living,None,24.420720982445214,None
sqft_above,None,20.024722417091304,None
floors_square,None,0.0,None
yr_renovated,None,0.0,None
yr_built,None,0.0,None
sqft_basement,None,0.0,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [9]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [12]:
import numpy as np

In [13]:
l1_penalty_list = np.logspace(1,7,num=13)

In [14]:
models = []
for l1_penalty in l1_penalty_list:
    model = turicreate.linear_regression.create(training, features=all_features, target='price',
        l1_penalty=l1_penalty, l2_penalty=0, validation_set=None, verbose=False)
    models.append([l1_penalty, model])

In [15]:
def compute_rss(data, output, model):
    rss = ((output - model.predict(data))**2).sum()
    return rss

In [18]:
rss_list = []
for l1,model in models:
    rss = compute_rss(validation,validation['price'],model)
    print(f"RSS of model l1_penalty={l1:e}: {rss}")
    rss_list.append((rss,l1))
rss_list.sort()

RSS of model l1_penalty=1.000000e+01: 625766285142459.9
RSS of model l1_penalty=3.162278e+01: 625766285362394.1
RSS of model l1_penalty=1.000000e+02: 625766286057885.0
RSS of model l1_penalty=3.162278e+02: 625766288257224.8
RSS of model l1_penalty=1.000000e+03: 625766295212186.6
RSS of model l1_penalty=3.162278e+03: 625766317206080.5
RSS of model l1_penalty=1.000000e+04: 625766386760658.1
RSS of model l1_penalty=3.162278e+04: 625766606749278.5
RSS of model l1_penalty=1.000000e+05: 625767302791634.1
RSS of model l1_penalty=3.162278e+05: 625769507643886.2
RSS of model l1_penalty=1.000000e+06: 625776517727024.0
RSS of model l1_penalty=3.162278e+06: 625799062845467.0
RSS of model l1_penalty=1.000000e+07: 625883719085425.2


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [19]:
rss_list[0]

(625766285142459.9, 10.0)

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [40]:
print(models[0][0])
models[0][1].coefficients.sort('value',ascending=False).print_rows(18)

10.0
+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|    waterfront    |  None |  601905.5945452718   |  None  |
|       view       |  None |  93312.85731187189   |  None  |
|    bathrooms     |  None |  25409.58893412067   |  None  |
|      floors      |  None |  21204.33546695013   |  None  |
|   (intercept)    |  None |  18993.427212770577  |  None  |
|  floors_square   |  None |  12915.524336072436  |  None  |
|     bedrooms     |  None |  7936.967679031306   |  None  |
|    condition     |  None |  6609.035712447216   |  None  |
|      grade       |  None |  6206.939991880551   |  None  |
| sqft_living_sqrt |  None |  1124.6502128077207  |  None  |
| bedrooms_square  |  None |   936.993368193299   |  None  |
|  sqft_lot_sqrt   |  None |  148.2583910114082   |  None  |
|  sqft_basement   |  None |  122.36782753411931  |  None  |
|   yr_renovated   

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [41]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [42]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [45]:
models = []
for i,l1_penalty in enumerate(l1_penalty_values):
    model = turicreate.linear_regression.create(training, features=all_features, target='price',
        l1_penalty=l1_penalty, l2_penalty=0.0, validation_set=None, verbose=False)
    print(f"{i} l1_penalty: {l1_penalty}\n\tnonzero features: {model.coefficients['value'].nnz()}")

0 l1_penalty: 100000000.0
	nonzero features: 18
1 l1_penalty: 127427498.57031322
	nonzero features: 18
2 l1_penalty: 162377673.91887242
	nonzero features: 18
3 l1_penalty: 206913808.111479
	nonzero features: 18
4 l1_penalty: 263665089.87303555
	nonzero features: 17
5 l1_penalty: 335981828.6283788
	nonzero features: 17
6 l1_penalty: 428133239.8719396
	nonzero features: 17
7 l1_penalty: 545559478.1168514
	nonzero features: 17
8 l1_penalty: 695192796.1775591
	nonzero features: 17
9 l1_penalty: 885866790.4100832
	nonzero features: 16
10 l1_penalty: 1128837891.6846883
	nonzero features: 15
11 l1_penalty: 1438449888.2876658
	nonzero features: 15
12 l1_penalty: 1832980710.8324375
	nonzero features: 13
13 l1_penalty: 2335721469.0901213
	nonzero features: 12
14 l1_penalty: 2976351441.6313133
	nonzero features: 10
15 l1_penalty: 3792690190.7322536
	nonzero features: 6
16 l1_penalty: 4832930238.571753
	nonzero features: 5
17 l1_penalty: 6158482110.6602545
	nonzero features: 3
18 l1_penalty: 78475

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [46]:
l1_penalty_min = l1_penalty_values[14]
l1_penalty_max = l1_penalty_values[15]

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [47]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [51]:
models = []
for i,l1_penalty in enumerate(l1_penalty_values):
    model = turicreate.linear_regression.create(training, features = all_features, target = 'price',
        l1_penalty=l1_penalty, l2_penalty=0.0, validation_set=None, verbose=False)
    rss = compute_rss(validation, validation['price'], model)
    print(f"{i}. l1_penalty: {l1_penalty}, sparsity: {model.coefficients['value'].nnz()}, rss: {rss}")
    models.append((l1_penalty, model))

0. l1_penalty: 2976351441.6313133, sparsity: 10, rss: 966925692362084.5
1. l1_penalty: 3019316638.9524155, sparsity: 10, rss: 974019450084556.1
2. l1_penalty: 3062281836.2735176, sparsity: 10, rss: 981188367942452.8
3. l1_penalty: 3105247033.5946198, sparsity: 10, rss: 989328342459473.8
4. l1_penalty: 3148212230.915722, sparsity: 10, rss: 998783211265891.2
5. l1_penalty: 3191177428.236824, sparsity: 10, rss: 1008477167020094.0
6. l1_penalty: 3234142625.557926, sparsity: 10, rss: 1018298780553819.8
7. l1_penalty: 3277107822.8790283, sparsity: 10, rss: 1028247992205977.2
8. l1_penalty: 3320073020.2001305, sparsity: 8, rss: 1034616909232828.0
9. l1_penalty: 3363038217.5212326, sparsity: 8, rss: 1038554735941040.8
10. l1_penalty: 3406003414.8423347, sparsity: 8, rss: 1043237237871703.0
11. l1_penalty: 3448968612.163437, sparsity: 7, rss: 1046937488751711.1
12. l1_penalty: 3491933809.484539, sparsity: 7, rss: 1051147625612860.9
13. l1_penalty: 3534899006.805641, sparsity: 7, rss: 1055992735

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [53]:
print(models[11][0])
models[11][1].coefficients.sort('value',ascending=False)

3448968612.163437


name,index,value,stderr
(intercept),None,222253.19254432785,None
bathrooms,None,15873.957259267981,None
grade,None,2899.4202697498786,None
sqft_living_sqrt,None,690.1147733133256,None
bedrooms,None,661.7227177822587,None
sqft_living,None,32.41022145125964,None
sqft_above,None,30.011575302201045,None
yr_renovated,None,0.0,None
yr_built,None,0.0,None
sqft_basement,None,0.0,None
